In [1]:
import pandas as pd
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from sklearn.base import BaseEstimator


In [2]:
df = pd.read_csv('../data/SBER_250914_250915.csv', sep=';')

df["date_str"]=df["<DATE>"].apply(lambda x: str(x).zfill(6)).astype(str)
df["time_str"]=df["<TIME>"].apply(lambda x: str(x).zfill(6)).astype(str)
df["datetime_str"]=df["date_str"]+df["time_str"]
df["ds"]= pd.to_datetime(df["datetime_str"], format='%y%m%d%H%M%S', utc=True)
df["unique_id"] = "SBER"

df.drop(["date_str", "time_str", "datetime_str", "<DATE>", "<TIME>", "<OPEN>", "<HIGH>", "<LOW>", "<VOL>"], axis=1, inplace=True)
df.rename(columns={"<CLOSE>": "y"}, inplace=True)

df = df.tail(200)

In [3]:
class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X['lag1']

In [4]:
from app.models.nixtla_models import keys_to_ml_models

model_keys = [
    'lgbm',
    'lasso',
    'lin_reg',
    'ridge',
    'knn',
    'mlp',
    'rf'
]

models = keys_to_ml_models(model_keys)
models

{'lgbm': LGBMRegressor(verbosity=-1),
 'lasso': Lasso(),
 'lin_reg': LinearRegression(),
 'ridge': Ridge(),
 'knn': KNeighborsRegressor(),
 'mlp': MLPRegressor(),
 'rf': RandomForestRegressor()}

In [5]:
mlf = MLForecast(
    models=models,
    freq='min',
    lags=[1],
    target_transforms=[Differences([1])],
)
mlf.fit(df=df)

forecast_df = mlf.predict(h=1)
forecast_df

,unique_id,ds,lgbm,lasso,lin_reg,ridge,knn,mlp,rf
0,SBER,2025-09-15 15:08:00+00:00,302.575349,302.551465,302.573419,302.5642,302.55,302.535771,302.586985


In [6]:
from app.models.nixtla_models import df_to_forecast_items

items = df_to_forecast_items(forecast_df, model_keys)
items


[ForecastItem(model='lgbm', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.5753491162005),
 ForecastItem(model='lasso', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.55146464646464),
 ForecastItem(model='lin_reg', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.5734194110513),
 ForecastItem(model='ridge', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.564199824433),
 ForecastItem(model='knn', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.55),
 ForecastItem(model='mlp', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.5357708811553),
 ForecastItem(model='rf', unique_id='SBER', ds=Timestamp('2025-09-15 15:08:00+0000', tz='UTC'), value=302.5869847619048)]